In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('taxi-analysis').getOrCreate()

In [5]:
import pandas as pd

In [7]:
zone_url = 'https://raw.githubusercontent.com/fivethirtyeight/uber-tlc-foil-response/master/uber-trip-data/taxi-zone-lookup.csv'
pd_zone_df = pd.read_csv(zone_url)
zone_df = spark.createDataFrame(pd_zone_df)
zone_df.limit(5).show()

+----------+-------------+--------------------+
|LocationID|      Borough|                Zone|
+----------+-------------+--------------------+
|         1|          EWR|      Newark Airport|
|         2|       Queens|         Jamaica Bay|
|         3|        Bronx|Allerton/Pelham G...|
|         4|    Manhattan|       Alphabet City|
|         5|Staten Island|       Arden Heights|
+----------+-------------+--------------------+



In [19]:
trip_files = r'C:\Users\kang\Desktop\data\*.csv'
trip_df = spark.read.csv(trip_files, inferSchema=True, header=True)
trip_df.limit(5).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [18]:
trip_df.printSchema()
zone_df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)



In [13]:
trip_df.createOrReplaceTempView('trips')
zone_df.createOrReplaceTempView('zone')

In [16]:
query = '''
SELECT 
    t.hvfhs_license_num,
    TO_DATE(t.pickup_datetime) as pickup_date,
    TO_DATE(t.dropoff_datetime) as dropoff_date,
    HOUR(t.pickup_datetime) as pickup_time,
    TO_DATE(t.dropoff_datetime) as dropoff_time,
    pz.Zone as pickup_zone,
    dz.Zone as dropoff_zone
FROM
    trips t
    LEFT JOIN
         zone pz
    ON
        t.PULocationID = pz.LocationID
    LEFT JOIN 
        zone dz
    ON
        t.DOLocationID = dz.LocationID
'''
comb_df = spark.sql(query)
comb_df.createOrReplaceTempView('comb')

In [17]:
comb_df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- pickup_date: date (nullable = true)
 |-- dropoff_date: date (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- dropoff_time: date (nullable = true)
 |-- pickup_zone: string (nullable = true)
 |-- dropoff_zone: string (nullable = true)



In [20]:
comb_df.select('pickup_time').describe().show()

+-------+------------------+
|summary|       pickup_time|
+-------+------------------+
|  count|          13392904|
|   mean|13.415796678599353|
| stddev| 6.318363050974371|
|    min|                 0|
|    max|                23|
+-------+------------------+

